In [1]:
import os
import sys
from pathlib import Path

import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import skimage.io

from collections import defaultdict
from tqdm.auto import tqdm
from joblib import Parallel, delayed
import re
import pickle

import networkx as nx
from sklearn.model_selection import train_test_split

In [2]:
import torch_geometric.utils
import torch

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
p_dir = (Path().cwd().parents[0]).absolute()

module_path = str(p_dir / "src")

if module_path not in sys.path:
    sys.path.append(module_path)

In [5]:
data_dir = (Path().cwd().parents[0] / 'data').absolute()

In [6]:
import lightning.pytorch as pl
import PPIGraph

# Load dataset

In [7]:
from torch_geometric.loader import DataLoader

# Define condition mapping
condition_mapping = {'HCC827Ctrl': 0, 'HCC827Osim': 1}

# Load graph dataset and process if neede
graph_path = data_dir / '9PPI_3D' / 'graphs' 

dataset = PPIGraph.GraphDataset2D3D(graph_path, 'raw', '2D3D_pt', condition_mapping=condition_mapping, n_c=2)
train_set, val_set, test_set = PPIGraph.train_test_val_split(dataset)

# Create Dataloader
train_loader = DataLoader(train_set, batch_size=32, shuffle=True)
val_loader = DataLoader(val_set, batch_size=32, shuffle=True)
test_loader = DataLoader(test_set, batch_size=32, shuffle=False)


In [8]:
# Print some information on the dataset
print(f'Dataset: {dataset}:')
print('======================')
print(f'Number of graphs: {len(dataset)}')
print(f'Number of features: {dataset.num_features}')
print(f'Number of classes: {dataset.num_classes}')

print(f'Train set: {len(train_set)}, test set: {len(test_set)}, val set: {len(val_set)}')
for step, data in enumerate(train_loader):

    print(f'Step {step + 1}:')
    print('=======')
    print(f'Number of graphs in the current batch: {data.num_graphs}')
    print(data)
    print()
    break

Dataset: GraphDataset2D3D(1451):
Number of graphs: 1451
Number of features: 9
Number of classes: 2
Train set: 697, test set: 580, val set: 174
Step 1:
Number of graphs in the current batch: 32
DataBatch(edge_index=[2, 94400], pos=[7055, 2], labels=[7055, 9], nuclei=[7055], weight=[94400], condition=[32], fov=[32], id=[32], pos3D=[7055, 3], edge_attr=[94400, 1], x=[7055, 9], y=[32], edge_weight_2D=[94400], edge_weight_3D=[94400], name=[32], batch=[7055], ptr=[33])



In [9]:
# # Print some information on the dataset
# print(f'Dataset: {dataset}:')
# print('======================')
# print(f'Number of graphs: {len(dataset)}')
# print(f'Number of features: {dataset.num_features}')
# print(f'Number of classes: {dataset.num_classes}')

# print(f'Train set: {len(train_set)}, test set: {len(test_set)}, val set: {len(val_set)}')
# for step, data in enumerate(train_loader):

#     print(f'Step {step + 1}:')
#     print('=======')
#     print(f'Number of graphs in the current batch: {data.num_graphs}')
#     print(data)
#     print()
#     break

In [10]:
data.edge_weight_2D

tensor([0.0369, 0.2073, 0.1594,  ..., 0.0978, 0.0774, 0.0679])

In [11]:
data.edge_weight_3D

tensor([6.4687e-01, 8.0602e-06, 9.1852e-04,  ..., 6.0540e-03, 8.5520e-03,
        8.7476e-03])

# K-fold on filter dataset

In [12]:
import itertools
from lightning.pytorch.accelerators import find_usable_cuda_devices
import wandb

In [13]:
from sklearn.model_selection import KFold, StratifiedKFold
from torch.utils.data import SubsetRandomSampler

In [14]:
# Filter out by maximum number of counts per cell
min_count = 100
max_count = 400

graph_path = data_dir / '9PPI_3D' / 'graphs' 
dataset = PPIGraph.GraphDataset2D3D(graph_path, 'raw', '2D3D_pt', condition_mapping=condition_mapping, n_c=2)

# Create Dataloader
loader = DataLoader(dataset, batch_size=1, shuffle=False)

# Get Indices
indices = []
for step, data in enumerate(loader):
    if len(data.x) <= min_count:
        continue 
    
    if (data.x.sum(axis=0) >= max_count).any():
        continue
    indices.append(step)
    
# Get subset dataset
dataset_filtered = dataset.index_select(indices)


In [15]:
len(dataset_filtered)

1298

In [16]:
dataset_name = '9PPI_2D3D_fusion'
condition = 'Kfold'
checkpoint_folder = (Path().cwd().parents[0]).absolute() / 'data' / '9PPI_3D'/"saved_models_fusion" / dataset_name / f"Graph_GNNs_{condition}" 
project_name = f'PLA_01122024_{dataset_name}_{condition}'

In [22]:
AVAIL_GPUS = [0]

# Setting the seed
pl.seed_everything(42)
epochs = 65

# NUM_LAYERS = 2
# HIDDEN_CHANNELS = 16
# pools = ['mean', 'max', 'sum', 'attention', 'attention2']
# num_layers = [2,3,4]
# hiddens = [16, 32, 64]


# model = 'GAT'
model = 'GINConv'
params = [
# [2,64,'attention','MLP'], 
# [2,64,'mean','GraphConv'],
# [2,32,'attention2','GCN'],
[4,32,'attention','GAT'],
[2,16,'mean','GINConv']
]

Global seed set to 42


In [23]:
params

[[4, 32, 'attention', 'GAT'], [2, 16, 'mean', 'GINConv']]

In [24]:
import PPIGraph
import torch
torch.set_float32_matmul_precision('medium')

In [25]:
fusion = 'bilinear'

In [26]:
k_folds = 5
kfold = KFold(n_splits=k_folds, shuffle=True, random_state=42)

for fold, (train_ids, valid_ids) in enumerate(kfold.split(dataset_filtered)):
    train_subset = dataset_filtered.index_select(train_ids.tolist())
    val_subset = dataset_filtered.index_select(valid_ids.tolist())
    for NUM_LAYERS, HIDDEN_CHANNELS, pool, model in params:
            # Path to the folder where the pretrained models are saved
        # CHECKPOINT_PATH = checkpoint_folder / f'GAT_{NUM_LAYERS}_{HIDDEN_CHANNELS}_onehot_{fold}' / pool
        CHECKPOINT_PATH = checkpoint_folder / f'{model}_{NUM_LAYERS}_{HIDDEN_CHANNELS}_{fusion}_{fold}' / pool
        CHECKPOINT_PATH.mkdir(parents=True, exist_ok=True)

        # Skip already trained kfold and pool
        checkpoint = CHECKPOINT_PATH / f"GraphLevel{model}" / f"GraphLevel{model}.ckpt" 
        if checkpoint.exists():
            print(checkpoint)
            continue

        # Run training
        run = wandb.init(project=project_name, name=f'{model}_{NUM_LAYERS}_{HIDDEN_CHANNELS}_{fusion}_{fold}', 
                        group=f'{model}_{fusion}', 
                        # mode="disabled"
                        )
        PPIGraph.train_graph_classifier_kfold_fusion(model, 
                                                train_subset, 
                                                val_subset, 
                                                dataset, 
                                                CHECKPOINT_PATH, 
                                                AVAIL_GPUS, 
                                                in_channels=9,
                                                hidden_channels=HIDDEN_CHANNELS, 
                                                out_channels = HIDDEN_CHANNELS,
                                                num_layers=NUM_LAYERS, 
                                                epochs=epochs,
                                                batch_size=64,
                                                fusion=fusion,
                                                graph_pooling=pool)
        run.finish()


Global seed set to 42
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Global seed set to 42
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

   | Name        | Type             | Params
--------------------------------------------------
0  | x_embedding | Identity         | 0     
1  | model2D     | GAT              | 14.1 K
2  | model3D     | GAT              | 14.1 K
3  | head        | BilinearFusion   | 107 K 
4  | pred        | Sequential       | 66    
5  | loss_module | CrossEntropyLoss | 0     
6  | train_acc   | BinaryAccuracy   | 0     
7  | train_auroc | BinaryAUROC      | 0     
8  | train_f1    | BinaryF1Score    | 0     
9  | valid_acc   | BinaryAccuracy   | 0     
10 | valid_auroc | BinaryAUROC      | 0     
11 | valid_f1    | BinaryF1Score    | 0     
12 | pool2D      | GlobalAttention  | 33    
13 | pool3D      | GlobalAttention  | 33    
---------------------------------

Using GAT


`Trainer.fit` stopped: `max_epochs=65` reached.


epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
lr-Adam,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train_acc,▁▅▅▅▆▇▆▇▇▇▇▇█▆▇▇▇▇█▇██▇▇██▇█▇█▇██▇▇████▇
train_auc,▁▄▅▅▆▆▆▆▆▇▇▇▇▆▇▇▇▇▇▇██▇▇▇▇▇▇▇█▇▇████▇▇█▇
train_f1,▁▂▆▁▄▆▆▆▆▇▆▆▇▅▇▆▇▇▇▆██▆▆▇▇▇▇▆█▇▇█▆█▇▇▇▇▇
train_loss_epoch,█▅▄▄▄▃▃▃▃▂▃▂▂▃▂▃▂▂▂▃▂▁▂▂▂▂▂▂▂▂▂▁▁▂▂▂▁▂▁▃
train_loss_step,█▅▂▇▃▃▁█▅▅▅▄▃▄▆▅▄▅█▃▅▆▅▃▅▄▇▂▄▂▃▃▅█▅▇▅▂▇▅
trainer/global_step,▁▁▁▂▁▂▂▁▃▃▂▃▂▂▄▂▂▄▂▂▅▂▂▅▂▆▆▂▆▆▃▇▃▃▇▃▃█▃▃
val_acc,▁▁▂▆▆▆▅▆▇▆█▅▄▆▅▄▇▇▅▅▆█▅▅▅▆▄▂▅▃▆▅▅▇▅▆█▇▅▅
val_auc,▁▂▄▄▅▆▆▇▆▆▅▇▆▅▆▆▇▇▅▆▅▆▆▆▇█▆▆▄████▅▇▅▇▇▆▇
val_f1,▁▁▂▆▆▅▄▆▇▆█▆▅▇▅▄▇▆▅▆██▆▅▅▅▄▂▆▄▆▅▅█▆▆█▆▅▅


Global seed set to 42
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Global seed set to 42
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

   | Name        | Type             | Params
--------------------------------------------------
0  | x_embedding | Identity         | 0     
1  | model2D     | GIN              | 1.1 K 
2  | model3D     | GIN              | 1.1 K 
3  | head        | BilinearFusion   | 19.6 K
4  | pred        | Sequential       | 34    
5  | loss_module | CrossEntropyLoss | 0     
6  | train_acc   | BinaryAccuracy   | 0     
7  | train_auroc | BinaryAUROC      | 0     
8  | train_f1    | BinaryF1Score    | 0     
9  | valid_acc   | BinaryAccuracy   | 0     
10 | valid_auroc | BinaryAUROC      | 0     
11 | valid_f1    | BinaryF1Score    | 0     
--------------------------------------------------
21.8 K    Trainable params
0         Non-trainable params
21.8 K    Tota

...

Global seed set to 42
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Global seed set to 42
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

   | Name        | Type             | Params
--------------------------------------------------
0  | x_embedding | Identity         | 0     
1  | model2D     | GIN              | 1.1 K 
2  | model3D     | GIN              | 1.1 K 
3  | head        | BilinearFusion   | 19.6 K
4  | pred        | Sequential       | 34    
5  | loss_module | CrossEntropyLoss | 0     
6  | train_acc   | BinaryAccuracy   | 0     
7  | train_auroc | BinaryAUROC      | 0     
8  | train_f1    | BinaryF1Score    | 0     
9  | valid_acc   | BinaryAccuracy   | 0     
10 | valid_auroc | BinaryAUROC      | 0     
11 | valid_f1    | BinaryF1Score    | 0     
--------------------------------------------------
21.8 K    Trainable params
0         Non-trainable params
21.8 K    Tota

GINConv


`Trainer.fit` stopped: `max_epochs=65` reached.
--- Logging error ---
Traceback (most recent call last):
  File "c:\Users\thu71\Anaconda3\envs\snowflake\lib\logging\__init__.py", line 1087, in emit
    self.flush()
  File "c:\Users\thu71\Anaconda3\envs\snowflake\lib\logging\__init__.py", line 1067, in flush
    self.stream.flush()
FileNotFoundError: [Errno 2] No such file or directory
Call stack:
  File "c:\Users\thu71\Anaconda3\envs\snowflake\lib\runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "c:\Users\thu71\Anaconda3\envs\snowflake\lib\runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "c:\Users\thu71\Anaconda3\envs\snowflake\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "c:\Users\thu71\Anaconda3\envs\snowflake\lib\site-packages\traitlets\config\application.py", line 1043, in launch_instance
    app.start()
  File "c:\Users\thu71\Anaconda3\envs\snowflake\lib\site-

--- Logging error ---
Traceback (most recent call last):
  File "c:\Users\thu71\Anaconda3\envs\snowflake\lib\site-packages\wandb\sdk\lib\sock_client.py", line 255, in _read_packet_bytes
    data = self._sock.recv(self._bufsize)
ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\thu71\Anaconda3\envs\snowflake\lib\site-packages\wandb\sdk\interface\router_sock.py", line 27, in _read_message
    resp = self._sock_client.read_server_response(timeout=1)
  File "c:\Users\thu71\Anaconda3\envs\snowflake\lib\site-packages\wandb\sdk\lib\sock_client.py", line 285, in read_server_response
    data = self._read_packet_bytes(timeout=timeout)
  File "c:\Users\thu71\Anaconda3\envs\snowflake\lib\site-packages\wandb\sdk\lib\sock_client.py", line 259, in _read_packet_bytes
    raise SockClientClosedError
wandb.sdk.lib.sock_client.SockCl

ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host